# DataVizHub Visualization Quickstart

This notebook demonstrates basic static, vector, particle, animation (frames), and interactive outputs using small in-repo samples.

Requirements:
- Static/animation: `pip install -e .[visualization]`
- Interactive (optional): `pip install -e .[interactive]`


In [ ]:
from pathlib import Path

import numpy as np

from zyra.visualization import (
    AnimateManager,
    ContourManager,
    HeatmapManager,
    VectorFieldManager,
    VectorParticlesManager,
)

out_dir = Path("samples/quickstart_outputs")
out_dir.mkdir(parents=True, exist_ok=True)

# Small demo arrays
arr = np.random.rand(10, 20).astype("float32")
ny, nx = 10, 20
U = np.ones((ny, nx), dtype="float32") * 0.5
V = np.zeros((ny, nx), dtype="float32")

extent = [-180, 180, -90, 90]

## Heatmap (static)

In [ ]:
hm = HeatmapManager(extent=extent)
hm.render(
    arr, width=400, height=200, dpi=100, colorbar=True, label="Value", units="arb"
)
heatmap_path = str(out_dir / "heatmap.png")
hm.save(heatmap_path)
heatmap_path

## Contour (static)

In [ ]:
cm = ContourManager(extent=extent, filled=True)
cm.render(arr, width=400, height=200, dpi=100, levels=8, colorbar=True)
contour_path = str(out_dir / "contour.png")
cm.save(contour_path)
contour_path

## Vector field (static quiver)

In [ ]:
vf = VectorFieldManager(extent=extent, density=0.3)
vf.render(u=U, v=V, width=400, height=200, dpi=100)
vector_path = str(out_dir / "vector.png")
vf.save(vector_path)
vector_path

## Particles (frames)

In [ ]:
# Build a tiny 3-frame stack by repeating vectors (for demo)
t = 3
U3 = np.repeat(U[None, ...], t, axis=0)
V3 = np.repeat(V[None, ...], t, axis=0)
np.save(out_dir / "u_stack.npy", U3)
np.save(out_dir / "v_stack.npy", V3)

vp = VectorParticlesManager(extent=extent)
vp.render(
    u=U3[0],
    v=V3[0],
    width=400,
    height=200,
    dpi=100,
    output_dir=str(out_dir / "frames_particles"),
    particles=50,
)
manifest_path = vp.save(str(out_dir / "frames_particles" / "manifest.json"))
manifest_path

## Animate heatmap (frames)

In [ ]:
stack = np.stack([arr, np.roll(arr, 1, axis=1), np.roll(arr, 2, axis=1)], axis=0)
np.save(out_dir / "stack.npy", stack)
anim = AnimateManager(
    mode="heatmap", extent=extent, output_dir=str(out_dir / "frames_heatmap")
)
anim.render(input_path=str(out_dir / "stack.npy"), cmap="YlOrBr", colorbar=True)
anim_manifest = anim.save(str(out_dir / "frames_heatmap" / "manifest.json"))
anim_manifest

## Interactive (optional)

In [ ]:
try:
    from zyra.visualization import InteractiveManager

    im = InteractiveManager(engine="folium", extent=extent)
    html = im.render(data=arr, mode="heatmap", cmap="YlOrBr", colorbar=True)
    inter_path = str(out_dir / "interactive_heatmap.html")
    im.save(inter_path)
    print(inter_path)
except Exception as e:
    print("Interactive extras not installed or failed:", e)